#คำอธิบาย

## Step 1 Load Data
- นำไฟล์ CSV ข้อมูลอาการผู้ป่วยเข้า Colab
- ใช้ `pandas.read_csv` อ่านข้อมูลเป็น DataFrame
- ข้อมูลนี้จะใช้สร้างโมเดลและประเมินผล

## Step 2-4 Data Preparation
- แปลงรายการอาการเป็น **multi-hot encoding** เพื่อใช้คำนวณความสัมพันธ์
- ทำ **cleaning & normalization** เช่น ลบ whitespace, lowercase
- เตรียมข้อมูลสำหรับสร้าง co-occurrence, confidence, lift, และ Jaccard similarity

## Step 5-7 Build Model
- คำนวณ **co-occurrence matrix** → นับจำนวนครั้งที่อาการเกิดร่วมกัน
- คำนวณ **confidence, lift** สำหรับ rule-based recommendation
- คำนวณ **Jaccard similarity** สำหรับ KNN-based recommendation
- สร้าง **prior vector** ตามเพศและกลุ่มอายุ

## Step 8 Evaluate Model (Train/Test Split)
- แบ่ง dataset เป็น **train/test** (เช่น 80/20)
- สร้างโมเดลจาก train set
- ประเมินบน test set ด้วย Leave-One-Out
- เก็บ **evaluation metrics**: Precision@K, Recall@K, MAP@K, Coverage
- **ตัวแปร `metrics_tt`** จะเก็บผลลัพธ์นี้ไว้สำหรับขั้นตอนต่อไป

## Step 9 Save Artifacts
- บันทึกไฟล์โมเดลและอาร์ติแฟกต์ เช่น `symptom_vocab.json`, `conf.npy`, `priors.json`, `config.json`
- สามารถเลือกเซฟลง **Google Drive** เพื่อเก็บถาวร
- โฟลเดอร์ตัวอย่าง: `/content/drive/MyDrive/agnos-rag/Tesk2/`

## Step 10-11 Load Artifacts and Recommend Function
- โหลดไฟล์ที่เซฟจาก Drive
- นิยามฟังก์ชัน `recommend(gender, age, input_symptoms, top_k)`
- ฟังก์ชันนี้จะ:
  1. Map อาการที่กรอกกับ vocabulary
  2. คำนวณคะแนนจาก **rule-based**, **KNN-based**, และ **prior**
  3. เลือก Top-K อาการที่แนะนำ
  4. แสดงเหตุผล (`why`) ว่าแต่ละอาการมาจากอาการอินพุตตัวไหน

## Step 12 Interactive Recommendation Form
- ใช้ `ipywidgets` สร้างฟอร์ม interactive
  - Dropdown เลือก **Gender**
  - Slider เลือก **Age**
  - Text box กรอก **Symptoms** (คั่นด้วย comma)
- ด้านบนช่อง Text มี **Label ตัวอย่าง**: `เช่น: ไอ, น้ำมูกไหล, เจ็บคอ`
- เมื่อกรอกข้อมูล → ระบบแนะนำ **Top-K Symptoms พร้อม score และ why**
- แสดง **evaluation metrics (`metrics_tt`)** จากขั้นตอน 8
- ทำให้ผู้ใช้เห็นผลลัพธ์และคุณภาพของโมเดลพร้อมกัน

---

### Notes
- **Top-K Recommendation** ใช้สำหรับช่วยผู้ใช้ประเมินอาการที่อาจเกิดร่วม
- **Evaluation Metrics** แสดงคุณภาพของโมเดลบน test set จริง
- Workflow นี้สามารถทำงานทั้งแบบ **script** และ **interactive** ใน Colab


## Modeling Approach & Techniques

### What Model/Method Was Used?
- โจทย์นี้ใช้ **Association Rule–based Recommendation + Similarity–based Method**  
  - **Association Rule (Conf, Lift)** คำนวณจาก **co-occurrence matrix** เพื่อนำมาใช้เป็นกฎ (เช่น ถ้ามี "ไอ" ก็มักจะมี "เสมหะ")  
  - **K-Nearest Neighbor (Jaccard Similarity)** ใช้ความคล้ายกันระหว่างอาการ (symptoms) เพื่อหาว่าอาการไหนมักจะปรากฏร่วมกันในกลุ่มผู้ป่วย  
  - **Demographic Prior** เพิ่มความน่าเชื่อถือโดยปรับตามเพศและอายุ เช่น บางอาการพบได้มากในเด็ก แต่ไม่ค่อยพบในผู้ใหญ่  

### Why This Approach?
1. **ความเหมาะสมกับข้อมูล (Data Characteristics)**  
   - ข้อมูลอาการเป็นแบบ *multi-label categorical* (ผู้ป่วย 1 คนมีหลายอาการพร้อมกัน)  
   - โมเดลเชิงสถิติที่อธิบายความสัมพันธ์แบบ *pairwise* เช่น **co-occurrence, confidence, lift** จึงเหมาะสมมากกว่าการใช้ classification ธรรมดา  

2. **การตีความง่าย (Interpretability)**  
   - เทคนิคอย่าง Confidence และ Lift มาจากแนวคิด **Association Rule Mining** ที่นักสถิติรู้จักกันดี (คล้าย Market Basket Analysis)  
   - สามารถอธิบายได้ว่าทำไมระบบถึงแนะนำอาการนั้น เช่น “เพราะมีโอกาส 0.25 ที่คนไอจะมีเสมหะ และ lift > 1.0”  

3. **ไม่ต้องใช้ Label แบบ Diagnosis**  
   - ปัญหานี้ไม่ใช่การ **predict โรค** แต่เป็นการ **recommend อาการ** → จึงไม่ต้องการ label ของโรค  
   - ใช้แค่ข้อมูล co-occurrence ของอาการก็เพียงพอ  

4. **Generalization Control**  
   - ใช้ **Train/Test Split Evaluation** เพื่อป้องกัน Overfitting  
   - Metrics อย่าง Precision@K, Recall@K, MAP@K, Coverage มาจากแนวคิดเชิงสถิติและ Information Retrieval ซึ่งเหมาะกับงาน recommendation  

### Perspective from Statistics
- วิธีนี้คือการผสม **Descriptive Statistics** (นับ co-occurrence, สร้าง contingency table) + **Inferential Logic** (confidence, lift) + **Predictive Component** (similarity, prior)  
- แทนที่จะสร้างโมเดลซับซ้อนแบบ Deep Learning → ใช้ **โมเดลเชิงสถิติที่โปร่งใสและตีความได้ง่าย** ซึ่งสำคัญมากสำหรับงานด้านสุขภาพ  
- จุดแข็งคือ **โปร่งใส, เข้าใจง่าย, ตีความได้**, และสามารถต่อยอดไปสู่ **Bayesian Modeling หรือ Logistic Regression** ได้ถ้ามีข้อมูลมากขึ้น  


# Formulas Used in This Project

### 1. Co-occurrence (การนับร่วม)
$$
\text{cooccur}(A,B) = \sum_{i=1}^{N} \mathbf{1}(A \in X_i \land B \in X_i)
$$

---

### 2. Support (การสนับสนุน)
$$
\text{support}(A,B) = \frac{\text{cooccur}(A,B)}{N}
$$

---

### 3. Confidence (ความมั่นใจ)
$$
\text{confidence}(A \Rightarrow B) = \frac{\text{cooccur}(A,B)}{\text{support}(A)}
$$

โดยที่
$$
\text{support}(A) = \frac{\text{count}(A)}{N}
$$

---

### 4. Lift (แรงยก)
$$
\text{lift}(A \Rightarrow B) = \frac{\text{confidence}(A \Rightarrow B)}{\text{support}(B)}
= \frac{P(A,B)}{P(A)\cdot P(B)}
$$

---

### 5. Jaccard Similarity (ความคล้ายกัน)
$$
J(A,B) = \frac{|A \cap B|}{|A \cup B|}
$$

---

### 6. Prior (ความน่าจะเป็นล่วงหน้า)
$$
P(B \mid \text{gender, age}) = \lambda \cdot P(B \mid \text{gender, age}) + (1-\lambda)\cdot P(B)
$$

---

### 7. Final Scoring Function (การรวมคะแนน)
$$
\text{score}(B) = \alpha \cdot \text{RuleScore}(B) + (1-\alpha) \cdot \text{KNNScore}(B)
$$  

$$
\text{final}(B) = \text{score}(B) \times \left(0.5 + 0.5 \cdot \text{prior}(B)\right)
$$

---

### 8. Evaluation Metrics (การวัดผลโมเดล)

- **Precision@K**
$$
\text{Precision@K} = \frac{\#\{\text{relevant items in Top-K}\}}{K}
$$

- **Recall@K**
$$
\text{Recall@K} = \frac{\#\{\text{relevant items in Top-K}\}}{\#\{\text{all relevant items}\}}
$$

- **MAP@K (Mean Average Precision)**
$$
MAP@K = \frac{1}{N} \sum_{i=1}^{N} \frac{1}{m_i} \sum_{k=1}^{K} P_i(k) \cdot rel_i(k)
$$  

- **Coverage**
$$
\text{Coverage} = \frac{\text{จำนวนอาการที่ระบบเคยแนะนำ}}{\text{จำนวนอาการทั้งหมดใน vocabulary}}
$$


#Step0 ไลบรารี

In [1]:
!pip -q install mlxtend rapidfuzz pythainlp fastapi uvicorn

import json, math, pickle, os, re
import numpy as np
import pandas as pd

from ast import literal_eval
from collections import defaultdict, Counter
from rapidfuzz import process, fuzz

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 49.1 MB/s eta 0:00:00


#Step1 ไฟล์

In [2]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

csv_path = list(uploaded.keys())[0]

df = pd.read_csv(csv_path)
df.head(2)

Saving [CONFIDENTIAL] AI symptom picker data (Agnos candidate assignment) - ai_symptom_picker.csv to [CONFIDENTIAL] AI symptom picker data (Agnos candidate assignment) - ai_symptom_picker.csv


,gender,age,summary,search_term
0,male,28,"{""diseases"": [], ""procedures"": [], ""no_symptom...","มีเสมหะ, ไอ"
1,male,27,"{""diseases"": [], ""procedures"": [], ""no_symptom...","ไอ, น้ำมูกไหล"


#Step2 แยก “อาการที่มีจริง” จากคอลัมน์ summary

In [3]:
 def safe_parse_json(x):
    try:
        return json.loads(x) if isinstance(x, str) and x.strip().startswith('{') else {}
    except Exception:
        return {}

def extract_yes_symptoms(summary_obj):
    out = []
    if isinstance(summary_obj, dict) and 'yes_symptoms' in summary_obj:
        items = summary_obj.get('yes_symptoms', [])
        for it in items:
            t = it.get('text', None)
            if isinstance(t, str) and len(t.strip())>0:
                out.append(t.strip())
    return out

df['summary_obj'] = df['summary'].apply(safe_parse_json)
df['raw_symptoms'] = df['summary_obj'].apply(extract_yes_symptoms)
df[['gender','age','raw_symptoms']].head(5)

,gender,age,raw_symptoms
0,male,28,"[เสมหะ, ไอ, การรักษาก่อนหน้า]"
1,male,27,"[ไอ, น้ำมูกไหล, การรักษาก่อนหน้า]"
2,female,26,"[ปวดท้อง, การรักษาก่อนหน้า]"
3,male,42,"[น้ำมูกไหล, การรักษาก่อนหน้า]"
4,female,40,"[ตาแห้ง, การรักษาก่อนหน้า]"


#Step3 ทำความสะอาดและทำให้เป็นรูปแบบเดียว (Normalization + Canonicalization)

In [4]:
noise_patterns = [
    r'^ประวัติ', r'การรักษา', r'previous treatment', r'โรคประจำตัว', r'ประวัติโรค', r'ยาที่ใช้อยู่'
]

synonym_map = {
    'เป็นไข้': 'ไข้',
    'ไข้สูง': 'ไข้',
    'fever': 'ไข้',
    'เจ็บคอ': 'เจ็บคอ',
    'ไอแห้ง': 'ไอ',
    'ไอมีเสมหะ': 'ไอ',
    'มีน้ำมูก': 'น้ำมูกไหล',
    'น้ำมูก': 'น้ำมูกไหล',
    'runny nose': 'น้ำมูกไหล',
    'ปวดหัว': 'ปวดศีรษะ',
    'เวียนหัว': 'เวียนศีรษะ',
    'บ้านหมุน': 'เวียนศีรษะ',
    'เหนื่อยหอบ': 'หายใจลำบาก'
}

def basic_norm(s):
    s2 = s.strip()
    s2 = re.sub(r'\s+', ' ', s2)
    return s2

def is_noise(s):
    t = s.lower()
    for pat in noise_patterns:
        if re.search(pat, t, flags=re.IGNORECASE):
            return True
    return False

def canonicalize_token(tok, vocab=None, threshold=90):
    t = basic_norm(tok)
    if t.lower() in synonym_map:
        return synonym_map[t.lower()]
    if vocab:
        match, score, _ = process.extractOne(t, list(vocab), scorer=fuzz.WRatio)
        if score >= threshold:
            return match
    return t

all_tokens = Counter()
for lst in df['raw_symptoms']:
    for s in lst:
        if not is_noise(s):
            all_tokens[ basic_norm(s) ] += 1

initial_vocab = set([w for w,cnt in all_tokens.items() if cnt>=2])

def normalize_row(sym_list, vocab):
    out = []
    for s in sym_list:
        if is_noise(s):
            continue
        can = canonicalize_token(s, vocab=vocab, threshold=90)
        out.append(can)
    uniq = sorted(list(set(out)))
    return uniq

df['symptoms'] = df['raw_symptoms'].apply(lambda lst: normalize_row(lst, initial_vocab))
df[['raw_symptoms','symptoms']].head(5)

,raw_symptoms,symptoms
0,"[เสมหะ, ไอ, การรักษาก่อนหน้า]","[เสมหะ, ไอ]"
1,"[ไอ, น้ำมูกไหล, การรักษาก่อนหน้า]","[น้ำมูกไหล, ไอ]"
2,"[ปวดท้อง, การรักษาก่อนหน้า]",[ปวดท้อง]
3,"[น้ำมูกไหล, การรักษาก่อนหน้า]",[น้ำมูกไหล]
4,"[ตาแห้ง, การรักษาก่อนหน้า]",[ตาแห้ง]


#Step4 ทำ Multi-hot matrix และกลุ่มอายุ

In [5]:
symptom_vocab = sorted(list(set(s for row in df['symptoms'] for s in row)))
sym2idx = {s:i for i,s in enumerate(symptom_vocab)}
idx2sym = {i:s for s,i in sym2idx.items()}
M = len(symptom_vocab)

def to_multihot(sym_list):
    x = np.zeros(M, dtype=np.uint8)
    for s in sym_list:
        if s in sym2idx:
            x[sym2idx[s]] = 1
    return x

X = np.vstack([to_multihot(lst) for lst in df['symptoms']])

def age_bin(a):
    if pd.isna(a): return 'unknown'
    a = int(a)
    if a<=12: return '0-12'
    if a<=18: return '13-18'
    if a<=29: return '19-29'
    if a<=44: return '30-44'
    if a<=59: return '45-59'
    return '60+'

df['age_bin'] = df['age'].apply(age_bin)
df['gender'] = df['gender'].fillna('unknown').str.lower()
N = X.shape[0]
M

142

#Step4.1 – Data Visualization

#Step5 สถิติร่วม, กฎร่วม และความคล้ายคลึงระหว่างอาการ

In [ ]:
support_counts = X.sum(axis=0).astype(np.int32)
cooccur = X.T @ X
np.fill_diagonal(cooccur, 0)

p_item = support_counts / N

conf = np.zeros_like(cooccur, dtype=float)
for i in range(M):
    denom = support_counts[i]
    if denom>0:
        conf[i,:] = cooccur[i,:] / denom

lift = np.zeros_like(cooccur, dtype=float)
for i in range(M):
    for j in range(M):
        if support_counts[i]>0 and p_item[j]>0:
            lift[i,j] = conf[i,j] / p_item[j]
        else:
            lift[i,j] = 0.0

jaccard = np.zeros_like(cooccur, dtype=float)
for i in range(M):
    for j in range(M):
        denom = support_counts[i] + support_counts[j] - cooccur[i,j]
        jaccard[i,j] = cooccur[i,j]/denom if denom>0 else 0.0

#Step6 Prior ตามประชากร (เพศ/อายุ)

In [ ]:
def priors_by_demo(df, X, group_cols=('gender','age_bin'), alpha=1.0):
    priors = {}
    global_freq = (X.sum(axis=0)+alpha) / (X.sum()+alpha*M)
    priors['global'] = global_freq.tolist()
    grp = df.groupby(list(group_cols)).indices
    for k, idxs in grp.items():
        xs = X[idxs]
        freq = (xs.sum(axis=0)+alpha) / (xs.sum()+alpha*M)
        priors[str(k)] = freq.tolist()
    return priors

priors = priors_by_demo(df, X)

#Step7 ฟังก์ชันแนะนำอาการ (ผสม Association + Item-KNN + Prior)

In [ ]:
alpha = 0.6
lam = 0.3
s_min = 3
c_min = 0.05

def get_demo_prior_vec(gender, age):
    g = (gender or 'unknown').lower()
    ab = age_bin(age)
    key = str((g, ab))
    vec_demo = np.array(priors.get(key, priors['global']), dtype=float)
    vec_global = np.array(priors['global'], dtype=float)
    mixed = lam*vec_demo + (1-lam)*vec_global
    return mixed

def map_inputs_to_vocab(inputs):
    mapped = []
    for s in inputs:
        if s in sym2idx:
            mapped.append(s)
        else:
            cand, score, _ = process.extractOne(s, list(sym2idx.keys()), scorer=fuzz.WRatio)
            mapped.append(cand)
    mapped = sorted(list(set(mapped)))
    return mapped

def recommend(gender, age, input_symptoms, top_k=5):
    S = map_inputs_to_vocab([basic_norm(s) for s in input_symptoms])
    if len(S)==0:
        prior_vec = get_demo_prior_vec(gender, age)
        top = np.argsort(-prior_vec)[:top_k]
        recs = []
        for j in top:
            recs.append({'symptom': idx2sym[j], 'score': float(prior_vec[j]), 'why': ['prior']})
        return {'recommended': recs, 'mapped_input': S}

    S_idx = [sym2idx[s] for s in S if s in sym2idx]
    cand_idx = [j for j in range(M) if j not in S_idx]

    rule_score = np.zeros(M, dtype=float)
    rule_explain = {j: [] for j in cand_idx}
    for a in S_idx:
        conf_row = conf[a,:]
        for b in cand_idx:
            if cooccur[a,b] >= s_min and conf_row[b] >= c_min and lift[a,b] > 1.0:
                rule_score[b] += conf_row[b] * math.log1p(lift[a,b])
                rule_explain[b].append((idx2sym[a], float(conf_row[b]), float(lift[a,b])))

    knn_score = np.zeros(M, dtype=float)
    for a in S_idx:
        knn_score += jaccard[a,:]

    prior_vec = get_demo_prior_vec(gender, age)

    final = alpha*rule_score + (1-alpha)*knn_score
    final = final * (0.5 + 0.5*prior_vec)

    order = [j for j in np.argsort(-final) if j in cand_idx]
    top = order[:top_k]

    recs = []
    for j in top:
        why_pairs = sorted(rule_explain[j], key=lambda x: x[1], reverse=True)[:2]
        why = [{'from': w[0], 'confidence': w[1], 'lift': w[2]} for w in why_pairs] if why_pairs else ['similarity/prior']
        recs.append({'symptom': idx2sym[j], 'score': float(final[j]), 'why': why})

    return {'recommended': recs, 'mapped_input': S}

#Step8 train/test evaluation

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import math

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

def build_model_from_train(train_df):
    X_train = np.vstack([to_multihot(lst) for lst in train_df['symptoms']])
    support_counts = X_train.sum(axis=0).astype(np.int32)
    cooccur = X_train.T @ X_train
    np.fill_diagonal(cooccur, 0)

    conf = np.zeros_like(cooccur, dtype=float)
    for i in range(len(support_counts)):
        if support_counts[i] > 0:
            conf[i,:] = cooccur[i,:] / support_counts[i]

    lift = np.zeros_like(cooccur, dtype=float)
    p_item = support_counts / len(train_df)
    for i in range(len(support_counts)):
        for j in range(len(support_counts)):
            if support_counts[i]>0 and p_item[j]>0:
                lift[i,j] = conf[i,j]/p_item[j]

    jaccard = np.zeros_like(cooccur, dtype=float)
    for i in range(len(support_counts)):
        for j in range(len(support_counts)):
            denom = support_counts[i] + support_counts[j] - cooccur[i,j]
            jaccard[i,j] = cooccur[i,j]/denom if denom>0 else 0

    priors_local = priors_by_demo(train_df, X_train)
    return support_counts, cooccur, conf, lift, jaccard, priors_local

def recommend_with_model(gender, age, input_symptoms, top_k,
                         support_counts, cooccur, conf, lift, jaccard, priors_local,
                         alpha=0.6, lam=0.3, s_min=3, c_min=0.05):
    S = map_inputs_to_vocab(input_symptoms)
    if len(S) == 0:
        prior_vec = get_demo_prior_vec(gender, age)
        top = np.argsort(-prior_vec)[:top_k]
        return [idx2sym[j] for j in top]

    S_idx = [sym2idx[s] for s in S]
    cand_idx = [j for j in range(M) if j not in S_idx]

    rule_score = np.zeros(M, dtype=float)
    for a in S_idx:
        conf_row = conf[a,:]
        for b in cand_idx:
            if cooccur[a,b] >= s_min and conf_row[b] >= c_min and lift[a,b] > 1.0:
                rule_score[b] += conf_row[b] * math.log1p(lift[a,b])

    knn_score = np.zeros(M, dtype=float)
    for a in S_idx:
        knn_score += jaccard[a,:]

    prior_vec = get_demo_prior_vec(gender, age)
    final = alpha*rule_score + (1-alpha)*knn_score
    final = final * (0.5 + 0.5*prior_vec)

    order = [j for j in np.argsort(-final) if j in cand_idx][:top_k]
    return [idx2sym[j] for j in order]

def evaluate_train_test(train_df, test_df, K=5, alpha=0.6, lam=0.3, s_min=3, c_min=0.05):
    support_counts, cooccur, conf, lift, jaccard, priors_local = build_model_from_train(train_df)

    rows = test_df[test_df['symptoms'].apply(lambda x: len(x)>=2)]
    hits = 0
    total = 0
    ap_sum = 0.0
    covered = set()

    for _, r in rows.iterrows():
        syms = r['symptoms']
        g, a = r['gender'], r['age']
        for held in syms:
            inputs = [s for s in syms if s != held]
            recs = recommend_with_model(g, a, inputs, K,
                                        support_counts, cooccur, conf, lift, jaccard, priors_local,
                                        alpha, lam, s_min, c_min)
            total += 1
            if held in recs:
                hits += 1
                rank = recs.index(held)+1
                ap_sum += 1.0/rank
            covered.update(recs)

    precision_at_k = hits/(len(rows)*np.mean([len(s) for s in rows['symptoms']])) if len(rows)>0 else 0
    recall_at_k = hits/total if total>0 else 0
    map_at_k = ap_sum/total if total>0 else 0
    coverage = len(covered)/M if M>0 else 0

    return {
        'Precision@K': precision_at_k,
        'Recall@K': recall_at_k,
        'MAP@K': map_at_k,
        'Coverage': coverage,
        'EvaluatedPairs': total
    }

metrics_tt = evaluate_train_test(train_df, test_df, K=5)
metrics_tt

#Step9 บันทึกอาร์ติแฟกต์สำหรับใช้งานใน API

In [ ]:
from google.colab import drive
import os, json, numpy as np

drive.mount('/content/drive')

save_dir = '/content/drive/MyDrive/agnos-rag/Tesk2'
os.makedirs(save_dir, exist_ok=True)

with open(os.path.join(save_dir, 'symptom_vocab.json'),'w',encoding='utf-8') as f:
    json.dump({'vocab': symptom_vocab}, f, ensure_ascii=False)

np.save(os.path.join(save_dir, 'support_counts.npy'), support_counts)
np.save(os.path.join(save_dir, 'cooccur.npy'), cooccur)
np.save(os.path.join(save_dir, 'jaccard.npy'), jaccard)
np.save(os.path.join(save_dir, 'conf.npy'), conf)
np.save(os.path.join(save_dir, 'lift.npy'), lift)

with open(os.path.join(save_dir, 'priors.json'),'w',encoding='utf-8') as f:
    json.dump(priors, f, ensure_ascii=False)

with open(os.path.join(save_dir, 'config.json'),'w',encoding='utf-8') as f:
    json.dump({'alpha': alpha, 'lambda': lam, 's_min': s_min, 'c_min': c_min}, f, ensure_ascii=False)

print("Saved artifacts to:", save_dir)

#Step10 โหลดอาร์ติแฟกต์จาก Google Drive

In [ ]:
import os, json, math, numpy as np
from rapidfuzz import process, fuzz

load_dir = '/content/drive/MyDrive/agnos-rag/Tesk2'

with open(os.path.join(load_dir, 'symptom_vocab.json'),'r',encoding='utf-8') as f:
    symptom_vocab = json.load(f)['vocab']

sym2idx = {s:i for i,s in enumerate(symptom_vocab)}
idx2sym = {i:s for s,i in sym2idx.items()}
M = len(symptom_vocab)

support_counts = np.load(os.path.join(load_dir, 'support_counts.npy'))
cooccur = np.load(os.path.join(load_dir, 'cooccur.npy'))
jaccard = np.load(os.path.join(load_dir, 'jaccard.npy'))
conf = np.load(os.path.join(load_dir, 'conf.npy'))
lift = np.load(os.path.join(load_dir, 'lift.npy'))

with open(os.path.join(load_dir, 'priors.json'),'r',encoding='utf-8') as f:
    priors = json.load(f)

with open(os.path.join(load_dir, 'config.json'),'r',encoding='utf-8') as f:
    cfg = json.load(f)

alpha = float(cfg['alpha'])
lam = float(cfg['lambda'])
s_min = int(cfg['s_min'])
c_min = float(cfg['c_min'])

#Step11 สร้างฟังก์ชัน Recommend ใหม่ (ใช้ไฟล์จาก Drive)

In [ ]:
def age_bin(a):
    if a is None: return 'unknown'
    a = int(a)
    if a<=12: return '0-12'
    if a<=18: return '13-18'
    if a<=29: return '19-29'
    if a<=44: return '30-44'
    if a<=59: return '45-59'
    return '60+'

def get_demo_prior_vec(gender, age):
    key = str(((gender or 'unknown').lower(), age_bin(age)))
    vec_demo = np.array(priors.get(key, priors['global']), dtype=float)
    vec_global = np.array(priors['global'], dtype=float)
    return lam*vec_demo + (1-lam)*vec_global

def map_inputs_to_vocab(inputs):
    out = []
    for s in inputs:
        if s in sym2idx:
            out.append(s)
        else:
            cand, score, _ = process.extractOne(s, list(sym2idx.keys()), scorer=fuzz.WRatio)
            out.append(cand)
    return sorted(list(set(out)))

def recommend(gender, age, input_symptoms, top_k=5):
    S = map_inputs_to_vocab(input_symptoms)
    if len(S)==0:
        prior_vec = get_demo_prior_vec(gender, age)
        top = np.argsort(-prior_vec)[:top_k]
        return {'recommended':[{'symptom': idx2sym[j], 'score': float(prior_vec[j])} for j in top],
                'mapped_input': S}

    S_idx = [sym2idx[s] for s in S]
    cand_idx = [j for j in range(M) if j not in S_idx]

    rule_score = np.zeros(M, dtype=float)
    rule_explain = {j: [] for j in cand_idx}
    for a in S_idx:
        conf_row = conf[a,:]
        for b in cand_idx:
            if cooccur[a,b] >= s_min and conf_row[b] >= c_min and lift[a,b] > 1.0:
                rule_score[b] += conf_row[b] * math.log1p(lift[a,b])
                rule_explain[b].append((idx2sym[a], float(conf_row[b]), float(lift[a,b])))

    knn_score = np.zeros(M, dtype=float)
    for a in S_idx:
        knn_score += jaccard[a,:]

    prior_vec = get_demo_prior_vec(gender, age)
    final = alpha*rule_score + (1-alpha)*knn_score
    final = final * (0.5 + 0.5*prior_vec)

    order = [j for j in np.argsort(-final) if j in cand_idx][:top_k]
    recs = []
    for j in order:
        why_pairs = sorted(rule_explain[j], key=lambda x: x[1], reverse=True)[:2]
        why = [{'from': w[0], 'confidence': w[1], 'lift': w[2]} for w in why_pairs] if why_pairs else ['similarity/prior']
        recs.append({'symptom': idx2sym[j], 'score': float(final[j]), 'why': why})
    return {'recommended': recs, 'mapped_input': S}

#Step12 ทดสอบการทำงานจาก Drive

In [ ]:
!pip install ipywidgets --quiet

In [ ]:
from ipywidgets import widgets, interact, VBox
import pandas as pd

def interactive_recommend(gender, age, symptoms):
    symptom_list = [s.strip() for s in symptoms.split(',')]

    test_input = recommend(gender=gender, age=age, input_symptoms=symptom_list, top_k=5)

    print("=== Recommended Symptoms ===")
    display(pd.DataFrame(test_input['recommended']))

    print("\n=== Evaluation Metrics (Train/Test Split) ===")
    display(metrics_tt)

label_symptoms = widgets.Label(
    value="กรอก Symptoms (คั่นแต่ละอาการด้วย comma) เช่น: ไอ, น้ำมูกไหล, เจ็บคอ"
)

gender_widget = widgets.Dropdown(
    options=['male','female'],
    value='female',
    description='Gender'
)

age_widget = widgets.IntSlider(
    value=26, min=0, max=100, step=1, description='Age'
)

symptoms_widget = widgets.Text(
    value='',
    description='Symptoms',
    placeholder='เช่น ไอ, น้ำมูกไหล'
)

form = VBox([label_symptoms, gender_widget, age_widget, symptoms_widget])

interact(
    interactive_recommend,
    gender=gender_widget,
    age=age_widget,
    symptoms=symptoms_widget
)